<a href="https://colab.research.google.com/github/mahmoodtt1/OR/blob/main/Streetcar_7_15_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
import os
import re
import pandas as pd

def extract_downtown_runs(df_raw):
    runs = []
    current_run = []
    collecting = False

    for _, row in df_raw.iterrows():
        if str(row[1]).strip() == "Station Stop":
            if current_run:
                runs.append(pd.DataFrame(current_run))
                current_run = []
            collecting = True
            current_run.append(row)
        elif collecting:
            current_run.append(row)

    if current_run:
        runs.append(pd.DataFrame(current_run))
    return runs

def extract_date_from_filename(filename):
    match = re.search(r"(\d{1,2})\s+(\d{4})", filename)
    if match:
        month = int(match.group(1))
        year = int(match.group(2))
        return f"{year}-{month:02d}"
    return "UnknownDate"

def extract_tsp_type(df_raw):
    for val in df_raw.iloc[0]:
        if isinstance(val, str):
            if "Opticom" in val:
                return "Opticom"
            if "Vontas" in val:
                return "Vontas"
    return "Unknown"

def read_cleaned_downtown_data(folder_path):
    all_runs = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".xls") or filename.endswith(".xlsm"):
            file_path = os.path.join(folder_path, filename)
            try:
                xls = pd.ExcelFile(file_path)
                if "Downtown Data" in xls.sheet_names:
                    df_raw = xls.parse("Downtown Data", header=None)
                    runs = extract_downtown_runs(df_raw)

                    file_date = extract_date_from_filename(filename)
                    tsp_type = extract_tsp_type(df_raw)

                    for i, run_df in enumerate(runs):
                        run_df = run_df.reset_index(drop=True)

                        # Extract only necessary columns by index
                        cols_to_extract = [2, 7, 8]  # Intersection, Arrive, Depart
                        run_clean = run_df.iloc[:, cols_to_extract].copy()
                        run_clean.columns = ["Intersection", "Arrive", "Depart"]

                        # Add metadata
                        run_clean["TSP Type"] = tsp_type
                        run_clean["Date"] = file_date
                        run_clean["RunID"] = f"{file_date}_{tsp_type}_Run{i+1}"
                        run_clean["File"] = filename

                        all_runs.append(run_clean)

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    if all_runs:
        return pd.concat(all_runs, ignore_index=True)
    else:
        return pd.DataFrame(columns=["Intersection", "Arrive", "Depart", "TSP Type", "Date", "RunID", "File"])

# Example usage in Colab:
# folder_path = "/content/drive/My Drive/Colab Notebooks/tsp_data"
# df_clean = read_cleaned_downtown_data(folder_path)
# df_clean.head()


In [14]:

folder_path = "/content/drive/My Drive/Colab Notebooks/tsp_data"
df3 = read_downtown_data_from_folder(folder_path)

# Optional: preview
df3.head()

/tmp/ipython-input-1-3779080247.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(all_runs, ignore_index=True)


2,"Focus on data for stations (bold), intersection data use to note dwell at intersections.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vontas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,File,RunID
0,NaN,Intersection at,Union Station Crosswalk,NaT,NaN,NaN,NaN,11:46:32,11:46:35,NaN,NaN,00:00:03,NaN,NaN,NaN,"39° 5'6.39""N","94°35'2.46""W",Running Time Data for ONLY OPTICOM Mahmood 5 2...,Running Time Data for ONLY OPTICOM Mahmood 5 2...
1,NaN,Intersection at,20th St. & Main,NaT,NaN,NaN,NaN,11:47:29,11:47:40,NaN,NaN,00:00:11,NaN,NaN,NaN,"39° 5'20.83""N","94°35'1.34""W",Running Time Data for ONLY OPTICOM Mahmood 5 2...,Running Time Data for ONLY OPTICOM Mahmood 5 2...
2,NaN,Intersection at,19th St. & Main,NaT,NaN,NaN,NaN,11:48:05,11:48:08,NaN,NaN,00:00:03,NaN,NaN,NaN,"39° 5'25.53""N","94°35'1.12""W",Running Time Data for ONLY OPTICOM Mahmood 5 2...,Running Time Data for ONLY OPTICOM Mahmood 5 2...
3,NaN,Intersection at,18th St. & Main,NaT,NaN,NaN,NaN,11:49:00,11:49:02,NaN,NaN,00:00:02,NaN,NaN,NaN,"39° 5'30.39""N","94°35'0.85""W",Running Time Data for ONLY OPTICOM Mahmood 5 2...,Running Time Data for ONLY OPTICOM Mahmood 5 2...
4,NaN,Intersection at,16th St. & Main,NaT,NaN,NaN,NaN,11:49:27,11:49:29,NaN,NaN,00:00:02,NaN,NaN,NaN,"39° 5'40.50""N","94°35'0.44""W",Running Time Data for ONLY OPTICOM Mahmood 5 2...,Running Time Data for ONLY OPTICOM Mahmood 5 2...


In [40]:
df3["Vontas"].unique()


array([nan], dtype=object)

In [20]:
df3.columns.values[7] = "ArrivalTime"
df3.columns.values[8] = "DepartureTime"
df3.columns.values[2] = "intersection"
df3.head()

2,"Focus on data for stations (bold), intersection data use to note dwell at intersections.",NaN,intersection,NaN,NaN,NaN,NaN,ArrivalTime,DepartureTime,Vontas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,File,RunID
0,NaN,Intersection at,Union Station Crosswalk,NaT,NaN,NaN,NaN,11:46:32,11:46:35,NaN,NaN,00:00:03,NaN,NaN,NaN,"39° 5'6.39""N","94°35'2.46""W",Running Time Data for ONLY OPTICOM Mahmood 5 2...,Running Time Data for ONLY OPTICOM Mahmood 5 2...
1,NaN,Intersection at,20th St. & Main,NaT,NaN,NaN,NaN,11:47:29,11:47:40,NaN,NaN,00:00:11,NaN,NaN,NaN,"39° 5'20.83""N","94°35'1.34""W",Running Time Data for ONLY OPTICOM Mahmood 5 2...,Running Time Data for ONLY OPTICOM Mahmood 5 2...
2,NaN,Intersection at,19th St. & Main,NaT,NaN,NaN,NaN,11:48:05,11:48:08,NaN,NaN,00:00:03,NaN,NaN,NaN,"39° 5'25.53""N","94°35'1.12""W",Running Time Data for ONLY OPTICOM Mahmood 5 2...,Running Time Data for ONLY OPTICOM Mahmood 5 2...
3,NaN,Intersection at,18th St. & Main,NaT,NaN,NaN,NaN,11:49:00,11:49:02,NaN,NaN,00:00:02,NaN,NaN,NaN,"39° 5'30.39""N","94°35'0.85""W",Running Time Data for ONLY OPTICOM Mahmood 5 2...,Running Time Data for ONLY OPTICOM Mahmood 5 2...
4,NaN,Intersection at,16th St. & Main,NaT,NaN,NaN,NaN,11:49:27,11:49:29,NaN,NaN,00:00:02,NaN,NaN,NaN,"39° 5'40.50""N","94°35'0.44""W",Running Time Data for ONLY OPTICOM Mahmood 5 2...,Running Time Data for ONLY OPTICOM Mahmood 5 2...


In [36]:
df4 = df3[["intersection", "ArrivalTime", "DepartureTime", "Vontas", "File", "RunID"]].copy()
df4.tail()

2,intersection,ArrivalTime,DepartureTime,Vontas,File,RunID
3355,19th St. & Main,NaN,NaN,NaN,Running Time Data for TSP 7 15 2025.xlsm,Running Time Data for TSP 7 15 2025.xlsm_Run100
3356,20th St. & Main,NaN,NaN,NaN,Running Time Data for TSP 7 15 2025.xlsm,Running Time Data for TSP 7 15 2025.xlsm_Run101
3357,Union Station Crosswalk,NaN,NaN,NaN,Running Time Data for TSP 7 15 2025.xlsm,Running Time Data for TSP 7 15 2025.xlsm_Run101
3358,NaN,NaN,NaN,NaN,Running Time Data for TSP 7 15 2025.xlsm,Running Time Data for TSP 7 15 2025.xlsm_Run102
3359,NaN,NaN,NaN,NaN,Running Time Data for TSP 7 15 2025.xlsm,Running Time Data for TSP 7 15 2025.xlsm_Run102


In [37]:
df_cleaned = df4.dropna(subset=["ArrivalTime", "DepartureTime"]).copy()


In [42]:
df_cleaned = df_cleaned.reset_index(drop=True)  # ensure index is clean
loop_marker = "Union Station Crosswalk"
current_run = 1
loop_count = 0
run_cycles = []

for _, row in df_cleaned.iterrows():
    if row["intersection"] == loop_marker:
        loop_count += 1
        if loop_count > 1 and loop_count % 2 == 1:  # every 2nd time is a new run start
            current_run += 1
    run_cycles.append(current_run)

df_cleaned["RunCycle"] = run_cycles


In [47]:
df_cleaned.tail(1000)

2,intersection,ArrivalTime,DepartureTime,Vontas,File,RunID,RunCycle
1663,Union Station Crosswalk,13:25:37,13:25:39,NaN,Running Time Data for TSP 6 30 2025.xlsm,Running Time Data for TSP 6 30 2025.xlsm_Run101,45
1664,Union Station Crosswalk,09:08:20,09:08:22,NaN,Running Time Data for TSP 7 7 2025.xlsm,Running Time Data for TSP 7 7 2025.xlsm_Run1,46
1665,20th St. & Main,09:09:26,09:09:27,NaN,Running Time Data for TSP 7 7 2025.xlsm,Running Time Data for TSP 7 7 2025.xlsm_Run1,46
1666,19th St. & Main,09:09:43,09:09:45,NaN,Running Time Data for TSP 7 7 2025.xlsm,Running Time Data for TSP 7 7 2025.xlsm_Run1,46
1667,18th St. & Main,09:10:49,09:10:51,NaN,Running Time Data for TSP 7 7 2025.xlsm,Running Time Data for TSP 7 7 2025.xlsm_Run2,46
1668,16th St. & Main,09:11:23,09:11:24,NaN,Running Time Data for TSP 7 7 2025.xlsm,Running Time Data for TSP 7 7 2025.xlsm_Run2,46
1669,Truman Rd S. & Main,09:12:20,09:12:21,NaN,Running Time Data for TSP 7 7 2025.xlsm,Running Time Data for TSP 7 7 2025.xlsm_Run3,46
1670,Truman Rd N. & Main,09:12:25,09:12:26,NaN,Running Time Data for TSP 7 7 2025.xlsm,Running Time Data for TSP 7 7 2025.xlsm_Run3,46
1671,14th St. & Main,09:12:40,09:12:41,NaN,Running Time Data for TSP 7 7 2025.xlsm,Running Time Data for TSP 7 7 2025.xlsm_Run4,46
1672,13th St. & Main,09:13:11,09:13:49,NaN,Running Time Data for TSP 7 7 2025.xlsm,Running Time Data for TSP 7 7 2025.xlsm_Run4,46
